# Bifurcation equations {#sec-20230102153124}

This notebook uses $\LaTeX\newcommand{\D}{\mathrm{d}}\newcommand{\E}{\mathcal{E}}\newcommand{\reals}{\mathbb{R}}$ custom macros.

In this chapter, the bifurcation analysis of the perfect system is performed symbolically. The starting point is the symbolic expression of the energy $(u, \lambda) \mapsto \E(u, \lambda)$ that was derived in @sec-20230208081549.

To begin with, we perform the relevant initializations and retrieve the expression of the energy.

In [1]:
import itertools
import json
import lsk

from sympy import *

In [2]:
E2, E2_dot, E3 = symbols(r"\E_2 \dot{\E}_2 \E_3")
E_ij_dot, E_ijk = symbols(r"\dot{E}_{ij} E_{ijk}")

t, λ = symbols(r"t \lambda")
u, u_hat, w_hat = symbols(r"u \hat{u} \hat{w}")

In [3]:
v = IndexedBase("v")
w = IndexedBase("w")
ξ = IndexedBase(r"\xi")

In [4]:
__index_number = itertools.count()

def new_index():
    return Idx(r"i_{" + str(next(__index_number)) + "}")
    #return Symbol(r"i_{" + str(next(__index_number)) + "}")

In [5]:
with open("setting-up_the_computational_stage.out.json", "r") as f:
    data = json.load(f)
    
u_star = eval(data["u_star"])
E = eval(data["E"])

In [6]:
#| code-fold: true
lsk.display_latex_equation(r"u^\star(\lambda)", u_star)
lsk.display_latex_equation(r"\E(u, \lambda)", E)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Outline of the strategy

In this section, we show that, besides the fundamental branch $u^\ast(\lambda)$, other (bifurcated) equilibrium branches may pass through the critical point $(u_0, \lambda_0)$. The starting point is the characterization of an equilibrium by the stationarity of the energy, which defines all equilibrium branches as implicit functions, which can be expanded with respect to some perturbation parameter.

The approach adopted here relies on the Lyapunov–Schmidt decomposition (see below). Note that other approaches are possible, see e.g. @chak2018.

For a given value of $\lambda$ (close to the critical value $\lambda_0$), we seek an equilibrium point $u$ of the
system, such that

$$
\E_{,u}(u, \lambda; \hat{u}) = 0 \quad \text{for all} \quad \hat{u} \in U.
$$ {#eq-20230102025737}

In [7]:
res = (E.diff(u) * u_hat).expand()

The above equation reads: for all $\hat{u} \in U$

In [8]:
#| code-fold: true
lsk.display_latex_equation(res, 0)

<IPython.core.display.Math object>

Note that the above variational problem is of order 1 with respect to both $u$ and $\lambda$, as proved below (where we scale both $u$ and $\lambda$ by the $t$ and observe the degree in $t$).

In [9]:
res.subs({λ: t * λ, u: t * u}).series(t, 0, 2)

t*(-\E_2*\dot{u}_0*\hat{u}*\lambda + \E_2*\hat{u}*u) + O(t**2)

The equilibrium state $u$ is projected onto the orthogonal subspaces $V$ and $W$.
$$
u = u^\ast(\lambda) + \xi_i \, v_i + w, \quad \text{with} \quad w \in W.
$$ {#eq-20220902174235}

It follows from the orthogonality of $V$ and $W$ that $\langle v_i, w \rangle = 0$ for all $i=1, \ldots, m$. Analysis of the bifurcated branches therefore reduces to finding $\xi_1, \ldots, \xi_m \in \reals$ and $w \in W$, such that
$$
\E_{,u}[u^\ast(\lambda) + \xi_i \, v_i + w, \lambda; \hat{u}] = 0 \quad \text{for all} \quad \hat{u} \in U.
$$ {#eq-20230107164346}

The method proceeds in three steps

1. Eq. (@eq-20230107164346) is first expressed for $\hat{u} \in W$ which leads to a definition of $w$ as an implicit function of $\xi_1, \ldots, \xi_m$ and $\lambda$ (see @sec-20230102030010),
2. Eq. (@eq-20230107164346) is then expressed for $\hat{u} \in V$ which leads to a definition of $\lambda$ as an implicit function of $\xi_1, \ldots, \xi_m$ (see @sec-20230102030125),
3. finally, a parametrization $\eta$ of $\xi_1, \ldots, \xi_m$ is introduced and the Taylor expansion of $u$ and $\lambda$ with respect to $\eta$ is derived (see @sec-20230102030224).

## Elimination of $w$ {#sec-20230102030010}

We first test Eq. (@eq-20230102025737) with $\hat{w} \in W$
$$
\E_{,u}[u^\ast(\lambda) + \xi_i \, v_i + w, \lambda; \hat{w}] = 0, \quad \text{for all} \quad \hat{w} \in W.
$$ {#eq-20220825143616}

This defines $w$ as an implicit function of $\xi_1, \ldots, \xi_m$ and $\lambda$. The following asymptotic expansion about $\xi_1 = \cdots = \xi_m = 0$ and $\lambda = \lambda_0$ is postulated
$$
\begin{aligned}
w(\xi_1, \ldots, \xi_m, \lambda) ={} & \xi_i \, w_i + \bigl( \lambda - \lambda_0 \bigr) \, w_\lambda + \tfrac{1}{2} \xi_i \, \xi_j \, w_{ij}\\
&+ \tfrac{1}{2} \bigl( \lambda - \lambda_0 \bigr)^2 w_{\lambda\lambda} + \xi_i \, \bigl( \lambda - \lambda_0 \bigr) w_{i\lambda} + \cdots,
\end{aligned}
$$ {#eq-20230107172048}
where $w_i$, $w_\lambda$, $w_{ij}$, $w_{\lambda\lambda}$ and $w_{i\lambda}$ are elements of $W$ to be defined.

In what follows, Eq. (@eq-20230107172048) is plugged into Eq. (@eq-20220825143616), which then delivers an asymptotic expansion of the stationarity equation. We will nullify the terms of order up to 2 in $\xi_i$ and $\lambda$.

It is observed that the residual $\E_{,u}$ involves $\E_3(u, u, \hat{u})$ and $\E_4(u, u, u, \hat{u})$. Since all terms of $u$ are of order 1 or more in $\xi_i$ and $\lambda$, $\E_4(u, u, u, \hat{u})$ will not be considered below. However $\E_3(u, u, \hat{u})$ will need to be properly defined index-wise. Therefore, we need to defined two equivalent expressions of $u$ with two different set of indices.

In [30]:
def w_():
    i = new_index()
    j = new_index()
    k = new_index()
    l = new_index()
    λ_ = Idx("\lambda")
    return (t * ξ[i] * w[i] + λ * w[λ]
            + (t**2 * ξ[j] * ξ[k] * w[j, k] + 2 * λ * t * ξ[l] * w[l, λ_] + λ ** 2 * w[λ_, λ_]) / 2)

In [31]:
def u_():
    i = new_index()
    return u_star + t * ξ[i] * v[i] + w_()

In [32]:
def res_w_():
    out = res.subs(u_hat, w_hat)
    out = out.subs(u**3, u_() * u_() * u_())
    out = out.subs(u**2, u_() * u_())
    out = out.subs(u, u_())
    return out.expand()

In [33]:
res_w = res_w_()

We can then nullify the various coefficients of this polynomial expansion, starting with the terms of order 1 in $\xi_i$, which delivers the following variational problem

In [34]:
Eq(res_w.coeff(t, 1).subs(λ, 0), 0)

Eq(\E_2*\hat{w}*\xi[i_{115}]*v[i_{115}] + \E_2*\hat{w}*\xi[i_{116}]*w[i_{116}], 0)

for all $\hat{w} \in W$. Since $v_i \in V$, we have: $\E_2(v_i, \bullet) = 0$ and the above variational problem reduces to

$$
\E_2(w_i, \hat{w}) = 0 \quad \text{for all} \quad \hat{w} \in W.
$$

As argued in @sec-20230107173921, this leads to $w_i = 0$.

Similarly, we find the term or of order 1 in $\lambda$, and evaluate it at $\xi_1 = \cdots = \xi_m = 0$. Unfortunately, substitutions do not work with indexed quantities. Since all occurences of $\xi_i$ have been scaled by $t$ so as to keep track of the order of each term, a workaround is to set $t$ to 0. This delivers the following variational problem

In [35]:
res_w.coeff(λ, 1).subs(t, 0)

\E_2*\hat{w}*w[\lambda]

for all $\hat{w} \in W$. The same argument leads to $w_\lambda = 0$.

Similarly again, for the term of order 2 in $\lambda$

In [36]:
Eq(res_w.coeff(λ, 2).subs(t, 0), 0)

Eq(\E_2*\hat{w}*w[\lambda, \lambda]/2 + \E_3*\hat{w}*w[\lambda]**2/2 + \dot{\E}_2*\hat{w}*w[\lambda], 0)

for all $\hat{w} \in W$. Since it was found that $w_\lambda = 0$, the above problem again reduces to $
\E_2(w_{\lambda\lambda}, \hat{w}) = 0$ for all $\hat{w} \in W$ and $w_{\lambda\lambda} = 0$.

To sum up, we have found so far that $w_i = w_\lambda = w_{\lambda\lambda} = 0$. At this point, it is interesting to redefine the function `w_()`, taking into account all these simplifications.

In [37]:
def w_():
    i = new_index()
    j = new_index()
    k = new_index()
    l = new_index()
    return (t**2 / 2 * ξ[j] * ξ[k] * w[j, k] + λ * t * ξ[l] * w[l, Idx(r"\lambda")])

We can then re-evaluate the residual

In [38]:
res_w = res_w_()

The $\xi_i \, \xi_j$ terms deliver the following variational problem

In [39]:
Eq((2 * res_w.coeff(t, 2).subs(λ, 0)).expand(), 0)

Eq(\E_2*\hat{w}*\xi[i_{147}]*\xi[i_{148}]*w[i_{147}, i_{148}] + \E_3*\hat{w}*\xi[i_{135}]*\xi[i_{140}]*v[i_{135}]*v[i_{140}], 0)

for all $\hat{w} \in W$ and $\xi_1, \ldots, \xi_m \in \reals$ (small enough). This variational problem  should be understood as

$$
\xi_i \, \xi_j \, \bigl[ \E_2(w_{ij}, \hat{w}) + \E_3(v_i, v_j, \hat{w}) \bigr] = 0
\quad \text{for all} \quad \hat{w} \in W.
$$

for all $\hat{w} \in W$ and $\xi_1, \ldots, \xi_m$ *small enough*. Since the above expression is homogeneous in $\xi_i$, it must in fact vanish *for all* $\xi_1, \ldots, \xi_m$. The resulting variational problem then coincides with (@eq-20230107180410). The $\xi_i \, \xi_j$ term of the asymptotic expansion of $w$ is therefore exactly $w_{ij}$ defined in @sec-20230402152824.

Finally, the $\xi_i \, \lambda$ terms deliver the following variational problem

In [40]:
lhs = res_w.coeff(t, 1).coeff(λ, 1).subs(λ, 0).expand()
Eq(lhs, 0)

Eq(\E_2*\hat{w}*\xi[i_{149}]*w[i_{149}, \lambda] + \E_3*\dot{u}_0*\hat{w}*\xi[i_{135}]*v[i_{135}]/2 + \E_3*\dot{u}_0*\hat{w}*\xi[i_{140}]*v[i_{140}]/2 - \E_3*\dot{u}_0*\hat{w}*\xi[i_{145}]*v[i_{145}] + \dot{\E}_2*\hat{w}*\xi[i_{145}]*v[i_{145}], 0)

The above expression is written with standardized index names, delivering the following variational problem.

In [41]:
assert lhs.is_Add

i = Idx("i")
lhs2 = 0

for expr in lhs.expand().args:
    d = get_contraction_structure(expr)
    (old,), = d.keys()
    lhs2 += expr.subs(old, i)

Eq(lhs2, 0)

Eq(\E_2*\hat{w}*\xi[i]*w[i, \lambda] + \dot{\E}_2*\hat{w}*\xi[i]*v[i], 0)

for all $\hat{w} \in W$ and $\xi_1, \ldots, \xi_m \in \reals$. We recognize the variational problem (@eq-20230107180501). In other words, the $\xi_i \, \lambda$ term of the asymptotic expansion of $w$ is $w_{i\lambda}$ defined in @sec-20230402152824.

So far, we have found the following expansion of $w$

$$
w(\xi_1, \ldots, \xi_m, \lambda) = \tfrac{1}{2} \xi_i \, \xi_j \, w_{ij} + \xi_i \, \bigl( \lambda - \lambda_0 \bigr) w_{i\lambda} + \cdots,
$$

where $w_{ij}$ and $w_{i\lambda}$ are defined by Eqs. (@eq-20230107180410) and (@eq-20230107180501), respectively.

## Elimination of $\lambda$ {#sec-20230102030125}

We now test Eq. (@eq-20230102025737) with $\hat{u} = v_i$ ($i = 1, \ldots, m$)

$$
\E_{,u}[u^\ast(\lambda) + \xi_j \, v_j + w(\xi_1, \ldots, \xi_m, \lambda), \lambda; v_i] = 0, \quad \text{for all} \quad i = 1, \ldots, m.
$$ {#eq-20230110092313}

This now defines $\lambda$ as an implicit function of $\xi_1, \ldots, \xi_m$. The following asymptotic expansion
about $\xi_1 = \cdots = \xi_m = 0$ is postulated

$$
\lambda(\xi_1, \ldots, \xi_m) = \xi_i \, \lambda_i + \tfrac{1}{2} \, \xi_i \, \xi_j \, \lambda_{ij} + \ldots
$$ {#eq-20230110091031}

where $\lambda_i$ and $\lambda_{ij}$ are scalar coefficients to be defined below.

We first create the SymPy expressions for the asymptotic expansions of $(\xi_1, \ldots, \xi_m, \lambda) \mapsto w(\xi_1, \ldots, \xi_m, \lambda)$, $(\xi_1, \ldots, \xi_m) \mapsto \lambda(\xi_1, \ldots, \xi_m)$ and finally $(\xi_1, \ldots, \xi_m) \mapsto u(\xi_1, \ldots, \xi_m)$ see Eqs. (@eq-20230107172048), (@eq-20230110091031) and (@eq-20220902174235).

In [42]:
λ = IndexedBase(r"\lambda")

def λ_():
    i = new_index()
    j = new_index()
    k = new_index()
    return t * ξ[i] * λ[i] + t**2 / 2 * ξ[j] * ξ[k] * λ[j, k]

In [43]:
def w_():
    i = new_index()
    j = new_index()
    k = new_index()
    l = new_index()
    return (t**2 / 2 * ξ[j] * ξ[k] * w[j, k] + λ_() * t * ξ[l] * w[l])

In [44]:
def u_():
    i = new_index()
    return u_star + t * ξ[i] * v[i] + w_()

In [45]:
#| code-fold: true
lsk.display_latex_equation(r"w(\xi_1, \ldots, \xi_m, \lambda)", w_())
lsk.display_latex_equation(r"\lambda(\xi_1, \ldots, \xi_m)", λ_())
lsk.display_latex_equation(r"u(\xi_1, \ldots, \xi_m)", u_())

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Then, the above expansions are plugged into the general expression of $\E_{,u}(u, \lambda; \bullet)$ and tested with $\hat{v} \in V$, see Eq. (@eq-20230110092313).

In [47]:
i = new_index()
res_v = res.subs(u_hat, v[i]).subs(u**2, u_() * u_()).subs(u, u_()).subs(Symbol("\lambda"), λ_()).expand()

We get an asymptotic expansion of the residual with respect to $\xi_1, \ldots, \xi_m$. This residual must vanish for all $\xi_i$ sufficiently small. Therefore, all coefficients of the expansion `res_v` must be nullified at all orders and for all $\xi_i$ sufficiently small. Considering the expansion up to third order, we get after simplification only *two* bifurcation equations, since the terms of order 0 and 1 are identically null, as verified below.

In [49]:
assert res_v.coeff(t, 0) == 0
assert res_v.coeff(t, 1).subs(E2 * v[i], 0) == 0

Upon cosmetic substitutions (that are defined step-by-step!), the $\xi_i \, \xi_j$ terms deliver the first form of the **first bifurcation equation**

In [53]:
lhs = res_v.coeff(t, 2).expand()
lhs

\E_2*\xi[i_{215}]*\xi[i_{216}]*v[i_{196}]*w[i_{215}, i_{216}]/2 + \E_2*\xi[i_{217}]*\xi[i_{218}]*t**2*\lambda[i_{222}, i_{223}]*\xi[i_{222}]*\xi[i_{223}]/2 + t*\lambda[i_{221}]*\xi[i_{221}][i_{218}]*v[i_{196}]*w[i_{217}] + \E_3*\dot{u}_0*\lambda[i_{221}]*\xi[i_{197}]*\xi[i_{221}]*v[i_{196}]*v[i_{197}]/2 + \E_3*\dot{u}_0*\lambda[i_{221}]*\xi[i_{205}]*\xi[i_{221}]*v[i_{196}]*v[i_{205}]/2 - \E_3*\dot{u}_0*\lambda[i_{221}]*\xi[i_{213}]*\xi[i_{221}]*v[i_{196}]*v[i_{213}] + \E_3*\xi[i_{197}]*\xi[i_{205}]*v[i_{196}]*v[i_{197}]*v[i_{205}]/2 + \dot{\E}_2*\lambda[i_{221}]*\xi[i_{213}]*\xi[i_{221}]*v[i_{196}]*v[i_{213}]

In [ ]:
lhs = res_v.coeff(t, 2)
lhs = lhs.subs(E2 * v_i, 0)
lhs = lhs.subs(ξ_q * ξ_r * E3 * v_q * v_r, ξ_j * ξ_k * E3 * v_j * v_k)
lhs = lhs.subs(ξ_r * v_r, ξ_q * v_q)
lhs = lhs.subs(λ_p * ξ_p, λ_k * ξ_k)
lhs = lhs.subs(ξ_q * v_q, ξ_j * v_j)
lhs = lhs.subs(E3 * v_i * v_j * v_k, E_ijk)
lhs = lhs.subs(E2_dot * v_i * v_j, E_ij_dot)

(bif_eq_1 := Eq(lhs, 0))

where we have introduced the symbols $\dot{E}_{ij}$ and $E_{ijk}$, defined by Eqs. (@eq-20230125062505) and (@eq-20230125062510), respectively. Turning now to the term of third-order in $\xi_i$.

**A REPRENDRE ICI 30/05/2023 09:33**

In [ ]:
lhs = res_v.coeff(t, 3)
lhs = lhs.subs(E2 * v_i, 0)

lhs

The above bifurcation equations should be understood as Eqs. (@eq-20230115101130) and (@eq-20230115101147) below. Note that these equations are in fact *asymptotic expansions*. As such, they should hold for all $\xi_1, \ldots, \xi_m$ sufficiently small. Since both equations are homogeneous in $\xi_1, \ldots, \xi_m$, it is equivalent to require these to hold for all $\xi_1, \ldots, \xi_m \in \reals$. 

## Summary

The bifurcated equilibrium branch is postulated as follows (Lyapunov–Schmidt decomposition)
$$
u = u^\ast(\lambda) + \xi_i \, v_i + w, \quad \text{with} \quad w \in W.
$$
whith $w = w(\xi_1, \ldots, \xi_m, \lambda) \in W$ and $\lambda = \lambda(\xi_1, \ldots, \xi_m)$.

The Taylor expansion of $w$ reduces to
$$
w(\xi_1, \ldots, \xi_m, \lambda) = \tfrac{1}{2} \xi_i \, \xi_j \, w_{ij} + \xi_i \, \bigl( \lambda - \lambda_0 \bigr) w_{i\lambda} + \cdots,
$$
where $w_{ij}$ and $w_{i\lambda}$ are the solutions to the variational problems (@eq-20230107180410) and (@eq-20230107180501) introduced in @sec-20230402152824.

Similarly, the Taylor expansion of $\lambda$ reads
$$
\lambda(\xi_1, \ldots, \xi_m) = \xi_i \, \lambda_i + \tfrac{1}{2} \, \xi_i \, \xi_j \, \lambda_{ij} + \cdots,
$$
where $\lambda_i$ and $\lambda_{ij}$ are scalar coefficients that solve the following bifurcation equations
$$
\xi_j \, \xi_k \, \bigl[
\tfrac{1}{2} \E_3(v_j, v_k, \hat{v})
+ \lambda_k \, \dot{\E}_2(v_j, \hat{v})
\bigr] = 0
$$ {#eq-20230115101130}
and
$$
\begin{aligned}[b]
\xi_j \, \xi_k \, \xi_l \, \bigl[
\tfrac{1}{2} \E_3(v_j, w_{kl}, \hat{v})
+ \lambda_l \, \E_3(v_j, w_{k\lambda}, \hat{v})
+ \tfrac{1}{6} \E_4(v_j, v_k, v_l, \hat{v}) &\\
+ \tfrac{1}{2} \lambda_l \, \dot{\E}_2(w_{jk}, \hat{v})
+ \lambda_k \, \lambda_l \, \dot{\E}_2(w_{j\lambda}, \hat{v})
+ \tfrac{1}{2} \lambda_{kl} \, \dot{\E}_2(v_j, \hat{v}) &\\
+ \tfrac{1}{2} \lambda_l \, \dot{\E}_3(v_j, v_k, \hat{v})
+ \tfrac{1}{2} \lambda_k \, \lambda_l \, \ddot{\E}_2(v_j, \hat{v})
\bigr] &= 0,
\end{aligned}
$$ {#eq-20230115101147}
for all $\xi_1, \ldots, \xi_m \in \reals$.